# Publishing Feedback

### Setup

In [1]:
### Mount Notebook to Google Drive
from google.colab import drive
drive.mount('/content/drive')
# change the working directory to the Drive root
%cd /content/drive/My\ Drive/Colab\ Notebooks/intro-to-langsmith-main/notebooks/module_4_collecting_human_feedback

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/intro-to-langsmith-main/notebooks/module_4_collecting_human_feedback


In [2]:
!pip install --quiet -U langchain-google-genai langgraph langgraph-sdk langgraph-checkpoint-sqlite langsmith langchain-community langchain-core
!pip install --quiet notebook python-dotenv lxml scikit-learn pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216

In [3]:
!pip install --quiet python-dotenv

In [4]:
from dotenv import load_dotenv
load_dotenv(".env")

True

In [ ]:
# You can set them inline
import os
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Adding Feedback to a Run

You can add feedback to an existing Run programmatically as long as you know the run_id. Let's grab a run_id from the LangSmith UI and add it here.

In [5]:
run_id = "999bb6a9-9881-4eab-8153-f8382376b214"

Now, let's add some continuous feedback

In [6]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="sample-continuous",
  score=7.0,
  comment="This is a sample piece of continuous feedback",
)

Feedback(id=UUID('1fdb2ed8-09c5-4a6b-925d-2964491cde9d'), created_at=datetime.datetime(2025, 9, 2, 8, 3, 53, 311108, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 9, 2, 8, 3, 53, 311113, tzinfo=datetime.timezone.utc), run_id=UUID('999bb6a9-9881-4eab-8153-f8382376b214'), trace_id=None, key='sample-continuous', score=7.0, value=None, comment='This is a sample piece of continuous feedback', correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}, user_id=None, user_name=None), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

And now, let's add some categorical feedback too!

In [7]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="sample-categorical",
  value="no",
  comment="This is a sample piece of categorical feedback",
)

Feedback(id=UUID('15da40b5-0eb2-4a1b-a4a6-7d7881c8309f'), created_at=datetime.datetime(2025, 9, 2, 8, 4, 4, 81641, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 9, 2, 8, 4, 4, 81645, tzinfo=datetime.timezone.utc), run_id=UUID('999bb6a9-9881-4eab-8153-f8382376b214'), trace_id=None, key='sample-categorical', score=None, value='no', comment='This is a sample piece of categorical feedback', correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}, user_id=None, user_name=None), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

### Pre-Generating Run IDs for Feedback

![Generate_run_id](../images/generate_run_id.png)

Using LangChain, we offer the ability to pre-generate and define run IDs, before your code is invoked and the run ID is generated. With this functionality, you're able to access your run ID before initial generation, which can be useful for actions like sending feedback. The example below demonstrates this.

In [8]:
import uuid

pre_defined_run_id = uuid.uuid4()
pre_defined_run_id

UUID('b4f85c86-58db-4d36-b6f6-1afde6c26d51')

In [9]:
from langsmith import traceable

@traceable
def foo():
    return "This is a sample Run!"

We are passing in a config with our function call through `langsmith_extra` that contains our pre-defined run_id

In [10]:
foo(langsmith_extra={"run_id": pre_defined_run_id})

'This is a sample Run!'

Now we can directly create feedback on this run!

In [11]:
from langsmith import Client

ls_client = Client()

ls_client.create_feedback(pre_defined_run_id, "user_feedback", score=1)

Feedback(id=UUID('2a014ca5-f6f8-4698-8f25-bc8e2ed4be1e'), created_at=datetime.datetime(2025, 9, 2, 8, 7, 33, 787142, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 9, 2, 8, 7, 33, 787145, tzinfo=datetime.timezone.utc), run_id=UUID('b4f85c86-58db-4d36-b6f6-1afde6c26d51'), trace_id=None, key='user_feedback', score=1, value=None, comment=None, correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}, user_id=None, user_name=None), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

### Pre-signed Feedback URLs

![presigned url](../images/presigned_url.png)


This can also be helpful for pre-signed feedback URLs. You would want to use these when you can't expose API keys or other secrets to the client, e.g. in a web application. Using a pre-determined run_id LangSmith has an endpoint create_presigned_feedback_token which will create a URL for sending feedback, without the use of secrets required.

In [12]:
pre_signed_url_id = uuid.uuid4()
pre_signed_url_id

UUID('7dc93657-cc66-4b9b-aa09-f20a87c26c32')

In [13]:
pre_signed_url = client.create_presigned_feedback_token(pre_signed_url_id, "user_presigned_feedback")

print(pre_signed_url)

id=UUID('3cc13697-5f17-4645-ac15-de1adc9f8125') url='https://api.smith.langchain.com/feedback/tokens/3cc13697-5f17-4645-ac15-de1adc9f8125' expires_at=datetime.datetime(2025, 9, 2, 11, 10, 27, 3964, tzinfo=datetime.timezone.utc)


Here, we can see that even though we haven't created a run yet, we're still able to generate the feedback URL.

Now, let's invoke our chain so the run with that ID is created:

In [14]:
foo(langsmith_extra={"run_id": pre_signed_url_id})

'This is a sample Run!'

Then, once our run is created, we can use the feedback URL to send feedback:

In [15]:
import requests

url_with_score = f"{pre_signed_url.url}?score=1"

response = requests.get(url_with_score)

if response.status_code >= 200 and response.status_code < 300:
    print("Feedback submitted successfully!")
else:
    print("Feedback submission failed!")

Feedback submitted successfully!
